In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jd_comment_with_label/jd_comment_data.xlsx
/kaggle/input/jd-data/data.bin
/kaggle/input/stopword/stopwords.txt


In [2]:
# !cp /kaggle/input/jd-data/data.bin /kaggle/working/

In [3]:
import pandas as pd
import torch 
import re

#导入停用词
stopwords_path = '/kaggle/input/stopword/stopwords.txt'
with open(stopwords_path, 'r', encoding='utf-8' ) as f:
    stopwords = [line.strip() for line in f.readlines()]
    
# 构建正则表达式模式（匹配停用词边界）
stopwords_pattern = r'\b(' + '|'.join(re.escape(word) for word in stopwords) + r')\b'

#不需要分词，后续交给tokennizer处理
def remove_stopwords(text):
    # 使用正则表达式替换停用词为空字符串
    cleaned_text = re.sub(stopwords_pattern, '', str(text))
    # 合并多余空格
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    return cleaned_text

#读取
comments =[]
scores = []
df = pd.read_excel('/kaggle/input/jd_comment_with_label/jd_comment_data.xlsx', )
for index, row in df.iterrows():
    score_value = int(row['评分（总分5分）(score)'])
    comment_text = row['评价内容(content)']   
    # clear_stopwords = [comments for comments in comment_text if comments not in stopwords]
    
    # 清洗停用词（保留完整文本）
    cleaned_text = remove_stopwords(comment_text)
    comments.append(cleaned_text)
    scores.append(score_value)

print(len(comments))
print(len(scores))

71818
71818


In [4]:
print(comments[:30])
print(scores[:30])

['此用户未填写评价内容', '此用户未填写评价内容', '此用户未填写评价内容', '此用户未填写评价内容', '此用户未填写评价内容', '此用户未填写评价内容', '此用户未填写评价内容', '此用户未填写评价内容', '此用户未填写评价内容', '此用户未填写评价内容', '此用户未填写评价内容', '此用户未填写评价内容', '此用户未填写评价内容', '此用户未填写评价内容', '此用户未填写评价内容', '一般般一分钱一分货吧', '此用户未填写评价内容', '此用户未填写评价内容', '商品质量很好很满意配送速度快啊而且配送员态度也非常好。', '。。。', '此用户未填写评价内容', '此用户未填写评价内容', '刘慧敏提莫摸摸摸休息泽TCL退咯的一组婆婆破鼓规土局', '此用户未填写评价内容', '此用户未填写评价内容', '还好还好还好还好红红火火好很好好', '此用户未填写评价内容', '此用户未填写评价内容', '此用户未填写评价内容', '好好好好好好好好好好好好好很好']
[5, 5, 3, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]


In [5]:
# import matplotlib.pyplot as plt
# import numpy as np

# data_len = [c for c in comments]
# plt.hist(data_len, bins=10)
# plt.xticks(np.arange(0, max_len + 30, 30))
# plt.show()

In [6]:
# #赛选无关标点符号格式控制符号
# comments = [cmt.replace('。', ' ').replace('，',' ')  for cmt in comments]

# print(len(comments))

In [7]:
#保存字典
import pickle
with open("data.bin", 'wb') as f:
    pickle.dump((comments,scores), f)


In [8]:
import pickle
#加载字典
with open('data.bin','rb') as f:
    comments, labels = pickle.load(f)
    

In [9]:
#加载Google的bert中文模型
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained('google-bert/bert-base-chinese', num_labels=6)

#加载预训练模型配置参数
# from transformers import AutoConfig
# config = AutoConfig.from_pretrained('google-bert/bert-base-chinese')

#加载tokenizer
from transformers import AutoTokenizer
# 加载词典创建分词器
tokenizer=  AutoTokenizer.from_pretrained('google-bert/bert-base-chinese')

#冻结bert参数
# model.bert.trainable=False

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

2025-05-23 14:21:55.329992: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748010115.530279      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748010115.601694      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

In [10]:
def build_collate(tokenizer):
    def collate_fn(batch):
        comments = [item[0] for item in batch]
        labels = [item[1] for item in batch]

        #tokenizer转换,truncation = True,超过模型指定最大长度token序列，裁剪
        model_inputs = tokenizer(comments,return_tensors='pt',padding=True,truncation=True,max_length=256)

        input_ids = model_inputs['input_ids']
        attention_mask = model_inputs['attention_mask']
        label = torch.tensor(labels)
        return input_ids, attention_mask, label
    return collate_fn

In [11]:
import torch 
import torch.nn as nn
from tqdm import tqdm
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import  SummaryWriter

writer = SummaryWriter()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
dataset = list(zip(comments, labels))
dataloader = DataLoader(dataset, batch_size=64, shuffle=True, collate_fn=build_collate(tokenizer))

In [13]:
#构建模型，把模型注册到cuda
model.to(device)

#损失函数
criterion = nn.CrossEntropyLoss()

#优化器
optimizer = optim.Adam(model.parameters(), lr=1e-5)


#训练模型
train_loss_cnt = 0
model.train()
num_epochs = 5
for epoch in range(num_epochs):
    total_loss = 0
    tpbar = tqdm(dataloader)
    for input_ids, attention_mask, label in tpbar:
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        label = label.to(device)

        #前向传播
        out = model(input_ids,attention_mask)
        #计算损失
        loss = criterion(out.logits,label)

        #反向传播
        optimizer.zero_grad()
        loss.backward()

        #梯度裁剪
        torch.nn.utils.clip_grad_norm_(model.parameters(),max_norm=1.0)

        #更新参数
        optimizer.step()
        
        # 释放显存
        del input_ids, attention_mask, label, out
        torch.cuda.empty_cache()
        
        total_loss += loss.item()
        
        # if (i+1) % 200 == 0:
        tpbar.set_description(f'Epoch[{epoch+1}/{num_epochs}],loss:{loss.item():.4f}')
        #记录tensorboard跟踪记录
        writer.add_scalar('train_loss',loss.item(),train_loss_cnt)
        train_loss_cnt += 1

Epoch[5/5],loss:0.0180: 100%|██████████| 1123/1123 [18:54<00:00,  1.01s/it]


In [14]:
torch.save(model.state_dict(),'jd_model.bin')

In [15]:
#test

#加载字典
import pickle
with open('data.bin','rb') as f:
    comments, labels = pickle.load(f)

#测试数据
text = ['非常差劲',
      '买回来非常好用，但是又有点难用，说好用，又不好用，说不好用吧，又好用',
      '牛逼666非常垃圾',
      '这一坨真是给我来了个惊喜']
model_input = tokenizer(text, return_tensors='pt', padding=True, truncation=True).to(device)

#加载模型
model.eval()
# model = AutoModelForSequenceClassification.from_pretrained('google-bert/bert-base-chinese', num_labels=6)
model.load_state_dict(torch.load('jd_model.bin'))
model.to(device)

#模型推理
with torch.no_grad():
    predict = model(**model_input)

pred = torch.argmax(predict.logits,dim=1).tolist()  # 转换为列表
print(f'预测结果：{pred}')

预测结果：[5, 5, 5, 5]
